# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Vienna Multi-Sensor** </span><span style="font-width:bold; font-size: 3rem; color:#333;">Batch Inference</span>

## 🗒️ This notebook performs batch inference for all 9 Vienna sensors:

### Pipeline Overview:
1. **Load Models**: Download 9 trained models from Hopsworks Model Registry
2. **Fetch Features**: Get weather forecasts and lagged PM2.5 features for each sensor
3. **Make Predictions**: Generate 7-day PM2.5 forecasts for all sensors
4. **Visualizations**: 
   - Individual sensor forecast plots (predicted vs actual)
   - Multi-sensor comparison dashboard
   - Hindcast performance monitoring
5. **Store Results**: Save predictions to monitoring Feature Group

### 🎯 Sensors (9 total):
- Kendlerstraße 40, Hausgrundweg 23, AKH Ostringweg
- Gaudenzdorfer Gürtel, Belgradplatz, Floridsdorf Gerichtsgasse
- Taborstraße, Wehlistraße 366, Josef Redl Gasse

## <span style='color:#ff5f27'> 📝 Imports

In [20]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction
HopsworksSettings initialized!


In [21]:
import datetime
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import hopsworks
import json
import matplotlib.pyplot as plt
from mlfs.airquality import util
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# normalize to UTC midnight
today_utc = pd.Timestamp.now(tz='UTC').normalize()
today = today_utc.to_pydatetime()
tomorrow = today + datetime.timedelta(days = 1)
today= today - datetime.timedelta(days = 1)
today

datetime.datetime(2025, 11, 15, 0, 0, tzinfo=datetime.timezone.utc)

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [42]:
print("🔐 Connecting to Hopsworks...")
project = hopsworks.login()
fs = project.get_feature_store() 
mr = project.get_model_registry()

secrets = hopsworks.get_secrets_api()

print("✅ Connected to Hopsworks Feature Store & Model Registry")

# Load Vienna sensors configuration
print("\n📡 Loading Vienna sensors configuration...")
vienna_config_str = secrets.get_secret("VIENNA_SENSORS_CONFIG").value
vienna_config = json.loads(vienna_config_str)

country = vienna_config['country']
city = vienna_config['city']
all_sensors = vienna_config['sensors']
active_sensors = [s for s in all_sensors if s.get('status') == 'active']

print(f"\n📍 City: {city}, {country}")
print(f"📡 Active sensors: {len(active_sensors)}")
print(f"\n🎯 Will perform inference for {len(active_sensors)} sensors:")
for i, sensor in enumerate(active_sensors, 1):
    print(f"  {i}. {sensor['name']} ({sensor['street']})")

🔐 Connecting to Hopsworks...
2025-11-16 13:29:16,546 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-16 13:29:16,555 INFO: Initializing external client
2025-11-16 13:29:16,555 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 13:29:17,852 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1298582
✅ Connected to Hopsworks Feature Store & Model Registry

📡 Loading Vienna sensors configuration...

📍 City: Vienna, Austria
📡 Active sensors: 9

🎯 Will perform inference for 9 sensors:
  1. Kendlerstraße 40 (Umspannwerk) (Kendlerstrasse-40)
  2. Hausgrundweg 23, Gstr. 254 (Hausgrundweg-23)
  3. Allgemeines Krankenhaus, Ostringweg (AKH-Ostringweg)
  4. Umspannwerk Gaudenzdorfer Gürtel (Gaudenzdorfer-Guertel)
  5. Belgradplatz (Südostecke), Gstr.Nr. 816 (Belgradplatz)
  6. Floridsdorf, Gerichtsgasse 1a (Floridsdorf-Gerichtsgasse)
  7. Ecke Taborstraße - Glockengasse (Taborstrasse)
  8. Wehlistraße 366, Gstr.Nr.2157 (Wehlistrasse-366)
  9. Schafbergbad, Josef Redl Gasse 2 (Josef-Redl-Gasse)


---

## <span style="color:#ff5f27;">🪝 Download Models from Model Registry</span>

Download trained models and Feature Views for all 9 Vienna sensors.

In [43]:
# Dictionary to store models and feature views for each sensor
sensor_models = {}

print("="*80)
print("🔄 Downloading models for all sensors...")
print("="*80)

for i, sensor in enumerate(active_sensors, 1):
    sensor_name = sensor['name']
    sensor_street = sensor['street']
    model_name = f"vienna_pm25_model_{sensor_street.replace('-', '_').lower()}"
    
    print(f"\n📥 [{i}/{len(active_sensors)}] Loading model for: {sensor_name}")
    print(f"   Model name: {model_name}")
    
    try:
        model_version = 2
        if i == 4:
            model_version = 3
        # Retrieve model from Model Registry
        retrieved_model = mr.get_model(
            name=model_name,
            version=model_version,
        )
        
        # Get associated Feature View
        fv = retrieved_model.get_feature_view()
        
        # Download model artifacts to local directory
        saved_model_dir = retrieved_model.download()
        
        # Load XGBoost model
        xgb_model = XGBRegressor()
        xgb_model.load_model(saved_model_dir + "/model.json")
        
        # Store in dictionary
        sensor_models[sensor_street] = {
            'model': xgb_model,
            'feature_view': fv,
            'model_dir': saved_model_dir,
            'sensor_name': sensor_name,
            'sensor': sensor
        }
        
        print(f"   ✅ Model loaded successfully")
        
    except Exception as e:
        print(f"   ❌ Error loading model: {str(e)}")
        continue

print(f"\n{'='*80}")
print(f"✅ Successfully loaded {len(sensor_models)} models")
print(f"{'='*80}")

🔄 Downloading models for all sensors...

📥 [1/9] Loading model for: Kendlerstraße 40 (Umspannwerk)
   Model name: vienna_pm25_model_kendlerstrasse_40
2025-11-16 13:29:20,249 INFO: There is no parent information


Downloading: 0.000%|          | 0/318097 elapsed<00:00 remaining<?

   ✅ Model loaded successfullydirs, 1 files)... DONE

📥 [2/9] Loading model for: Hausgrundweg 23, Gstr. 254
   Model name: vienna_pm25_model_hausgrundweg_23
2025-11-16 13:29:22,620 INFO: There is no parent information


Downloading: 0.000%|          | 0/317499 elapsed<00:00 remaining<?

   ✅ Model loaded successfullydirs, 1 files)... DONE

📥 [3/9] Loading model for: Allgemeines Krankenhaus, Ostringweg
   Model name: vienna_pm25_model_akh_ostringweg
2025-11-16 13:29:25,130 INFO: There is no parent information


...

   ✅ Model loaded successfullydirs, 1 files)... DONE

📥 [8/9] Loading model for: Wehlistraße 366, Gstr.Nr.2157
   Model name: vienna_pm25_model_wehlistrasse_366
2025-11-16 13:29:35,688 INFO: There is no parent information


Downloading: 0.000%|          | 0/309974 elapsed<00:00 remaining<?

   ✅ Model loaded successfullydirs, 1 files)... DONE

📥 [9/9] Loading model for: Schafbergbad, Josef Redl Gasse 2
   Model name: vienna_pm25_model_josef_redl_gasse
2025-11-16 13:29:37,690 INFO: There is no parent information


Downloading: 0.000%|          | 0/317740 elapsed<00:00 remaining<?

   ✅ Model loaded successfullydirs, 1 files)... DONE

✅ Successfully loaded 9 models


In [44]:
# Display summary of loaded models
print("\n📊 Loaded Models Summary:")
print(f"{'Sensor':<50} {'Model Status'}")
print("-" * 70)
for street, data in sensor_models.items():
    print(f"{data['sensor_name']:<50} ✅ Ready")


📊 Loaded Models Summary:
Sensor                                             Model Status
----------------------------------------------------------------------
Kendlerstraße 40 (Umspannwerk)                     ✅ Ready
Hausgrundweg 23, Gstr. 254                         ✅ Ready
Allgemeines Krankenhaus, Ostringweg                ✅ Ready
Umspannwerk Gaudenzdorfer Gürtel                   ✅ Ready
Belgradplatz (Südostecke), Gstr.Nr. 816            ✅ Ready
Floridsdorf, Gerichtsgasse 1a                      ✅ Ready
Ecke Taborstraße - Glockengasse                    ✅ Ready
Wehlistraße 366, Gstr.Nr.2157                      ✅ Ready
Schafbergbad, Josef Redl Gasse 2                   ✅ Ready


---

## <span style="color:#ff5f27;">✨ Prepare Features for Inference</span>

Fetch weather forecasts and lagged PM2.5 features for all sensors.

In [45]:
# Retrieve Feature Groups
print("🔄 Retrieving Vienna Feature Groups...")

air_quality_fg = fs.get_feature_group(
    name='air_quality_vienna',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_vienna',
    version=1,
)

print(f"✅ Feature Group: {air_quality_fg.name}")
print(f"✅ Feature Group: {weather_fg.name}")

# Fetch weather forecast data (future dates)
print(f"\n🌤️  Fetching weather forecasts from {today}...")
weather_forecast_df = weather_fg.filter(weather_fg.date >= today).read()

print(f"✅ Weather forecast data: {len(weather_forecast_df)} rows")
print(f"   Date range: {weather_forecast_df['date'].min()} to {weather_forecast_df['date'].max()}")
print(f"   Unique sensors: {weather_forecast_df['street'].nunique()}")

# Fetch historical air quality data (for lagged features)
print(f"\n📊 Fetching historical PM2.5 data for lagged features...")
air_quality_df = air_quality_fg.read()

print(f"✅ Air quality data: {len(air_quality_df)} rows")
print(f"   Date range: {air_quality_df['date'].min()} to {air_quality_df['date'].max()}")
print(f"   Unique sensors: {air_quality_df['street'].nunique()}")

🔄 Retrieving Vienna Feature Groups...
✅ Feature Group: air_quality_vienna
✅ Feature Group: weather_vienna

🌤️  Fetching weather forecasts from 2025-11-15 00:00:00+00:00...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
✅ Weather forecast data: 63 rows
   Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00
   Unique sensors: 9

📊 Fetching historical PM2.5 data for lagged features...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.62s) 
✅ Air quality data: 27713 rows
   Date range: 2014-01-01 00:00:00+00:00 to 2025-11-16 00:00:00+00:00
   Unique sensors: 9


---

## <span style="color:#ff5f27;">🤖 Make Predictions for All Sensors</span>

For each sensor:
1. **Load lagged features** from sensor-specific Feature Group (`air_quality_with_lags_vienna_{sensor}`)
2. **Fetch weather forecast** for the sensor location
3. **Construct feature matrix** (weather + lagged PM2.5 features)
4. **Generate 7-day PM2.5 predictions** using the trained model

**Note:** Lagged features are read directly from Feature Groups created during training, not recalculated.

In [46]:
# Dictionary to store predictions for all sensors
all_predictions = []

print("="*80)
print("🤖 Making predictions for all sensors...")
print("="*80)

for i, (sensor_street, model_data) in enumerate(sensor_models.items(), 1):
    sensor_name = model_data['sensor_name']
    model = model_data['model']
    sensor = model_data['sensor']
    
    print(f"\n🔮 [{i}/{len(sensor_models)}] Predicting for: {sensor_name}")
    print(f"   Street ID: {sensor_street}")
    
    try:
        # STEP 1: Get lagged features from Feature Group
        fg_name = f"air_quality_with_lags_vienna_{sensor_street.replace('-', '_').lower()}"
        
        print(f"   Loading Feature Group: {fg_name}")
        
        try:
            sensor_lags_fg = fs.get_feature_group(name=fg_name, version=1)
            sensor_lags_df = sensor_lags_fg.read()
            sensor_lags_df = sensor_lags_df.sort_values('date', ascending=False)
            
            # Get the most recent row with lagged features
            if len(sensor_lags_df) == 0:
                print(f"   ⚠️  Feature Group is empty, skipping...")
                continue
                
            latest_lags = sensor_lags_df.iloc[0]
            lag_1_pm25 = latest_lags['lag_1_pm25']
            lag_2_pm25 = latest_lags['lag_2_pm25']
            lag_3_pm25 = latest_lags['lag_3_pm25']
            
            print(f"   ✅ Lagged features from Feature Group:")
            print(f"      Latest date: {latest_lags['date']}")
            print(f"      lag_1_pm25: {lag_1_pm25:.1f}, lag_2_pm25: {lag_2_pm25:.1f}, lag_3_pm25: {lag_3_pm25:.1f}")
            
        except Exception as e:
            print(f"   ⚠️  Failed to load Feature Group '{fg_name}': {str(e)}")
            print(f"   Falling back to manual calculation...")
            
            # Fallback: manually calculate if FG not available
            sensor_aq = air_quality_df[air_quality_df['street'] == sensor_street].copy()
            sensor_aq = sensor_aq.sort_values('date', ascending=False)
            
            if len(sensor_aq) < 3:
                print(f"   ⚠️  Not enough historical data, skipping...")
                continue
            
            lag_1_pm25 = sensor_aq.iloc[0]['pm25']
            lag_2_pm25 = sensor_aq.iloc[1]['pm25']
            lag_3_pm25 = sensor_aq.iloc[2]['pm25']
            print(f"   Calculated lags: {lag_1_pm25:.1f}, {lag_2_pm25:.1f}, {lag_3_pm25:.1f}")
        
        # STEP 2: Filter sensor-specific weather forecast
        sensor_weather = weather_forecast_df[weather_forecast_df['street'] == sensor_street].copy()
        sensor_weather = sensor_weather.sort_values('date')
        
        if len(sensor_weather) == 0:
            print(f"   ⚠️  No weather forecast data for {sensor_street}, skipping...")
            continue
        
        print(f"   Weather forecast days: {len(sensor_weather)}")
        
        # STEP 3: Construct feature matrix for prediction
        # Features order: weather features + lagged features
        features_list = []
        
        for idx, row in sensor_weather.iterrows():
            feature_dict = {
                'date': row['date'],
                'temperature_2m_mean': row['temperature_2m_mean'],
                'precipitation_sum': row['precipitation_sum'],
                'wind_speed_10m_max': row['wind_speed_10m_max'],
                'wind_direction_10m_dominant': row['wind_direction_10m_dominant'],
                'lag_1_pm25': lag_1_pm25,
                'lag_2_pm25': lag_2_pm25,
                'lag_3_pm25': lag_3_pm25,
            }
            features_list.append(feature_dict)
        
        features_df = pd.DataFrame(features_list)
        
        # STEP 4: Make predictions
        X_inference = features_df[['lag_1_pm25', 'lag_2_pm25', 'lag_3_pm25', 'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 
                                     'wind_direction_10m_dominant']]
        
        predictions = model.predict(X_inference)
        
        # Add predictions to dataframe
        features_df['predicted_pm25'] = predictions
        features_df['sensor_name'] = sensor_name
        features_df['street'] = sensor_street
        features_df['city'] = city
        features_df['country'] = country
        
        all_predictions.append(features_df)
        
        print(f"   ✅ Predictions generated: {len(predictions)} days")
        print(f"   Predicted PM2.5 range: {predictions.min():.1f} - {predictions.max():.1f}")
        
    except Exception as e:
        print(f"   ❌ Error making predictions: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Combine all predictions
if len(all_predictions) > 0:
    predictions_df = pd.concat(all_predictions, ignore_index=True)
    print(f"\n{'='*80}")
    print(f"✅ Predictions completed for {len(all_predictions)} sensors")
    print(f"   Total prediction rows: {len(predictions_df)}")
    print(f"{'='*80}")
else:
    print("\n❌ No predictions generated")

predictions_df.head(20)

🤖 Making predictions for all sensors...

🔮 [1/9] Predicting for: Kendlerstraße 40 (Umspannwerk)
   Street ID: Kendlerstrasse-40
   Loading Feature Group: air_quality_with_lags_vienna_kendlerstrasse_40
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.10s) 
   ✅ Lagged features from Feature Group:
      Latest date: 2025-11-16 00:00:00+00:00
      lag_1_pm25: 42.0, lag_2_pm25: 42.0, lag_3_pm25: 36.0
   Weather forecast days: 7
   ✅ Predictions generated: 7 days
   Predicted PM2.5 range: 44.0 - 65.3

🔮 [2/9] Predicting for: Hausgrundweg 23, Gstr. 254
   Street ID: Hausgrundweg-23
   Loading Feature Group: air_quality_with_lags_vienna_hausgrundweg_23
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.38s) 
   ✅ Lagged features from Feature Group:
      Latest date: 2025-11-16 00:00:00+00:00
      lag_1_pm25: 61.0, lag_2_pm25: 46.0, lag_3_pm25: 35.0
   Weather forecast days: 7
   ✅ Predictions generated: 7 days
   Predicted PM2.5 r

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,lag_1_pm25,lag_2_pm25,lag_3_pm25,predicted_pm25,sensor_name,street,city,country
0,2025-11-16 00:00:00+00:00,12.05,0.0,8.396570,149.036316,42.0,42.0,36.0,49.260567,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
1,2025-11-17 00:00:00+00:00,9.55,2.8,13.324863,271.548126,42.0,42.0,36.0,48.216461,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
2,2025-11-18 00:00:00+00:00,5.10,0.0,17.015474,276.072357,42.0,42.0,36.0,43.999825,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
3,2025-11-19 00:00:00+00:00,4.75,0.0,15.038350,137.910919,42.0,42.0,36.0,54.732391,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
4,2025-11-20 00:00:00+00:00,7.05,0.0,10.538843,262.146790,42.0,42.0,36.0,46.860004,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
5,2025-11-21 00:00:00+00:00,3.10,0.1,12.727921,298.739685,42.0,42.0,36.0,54.531956,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
6,2025-11-22 00:00:00+00:00,2.70,0.1,4.394360,55.007900,42.0,42.0,36.0,65.341042,Kendlerstraße 40 (Umspannwerk),Kendlerstrasse-40,Vienna,Austria
7,2025-11-16 00:00:00+00:00,11.50,0.0,8.311245,162.349792,61.0,46.0,35.0,65.814873,"Hausgrundweg 23, Gstr. 254",Hausgrundweg-23,Vienna,Austria
8,2025-11-17 00:00:00+00:00,9.65,3.2,5.815978,248.198532,61.0,46.0,35.0,65.903534,"Hausgrundweg 23, Gstr. 254",Hausgrundweg-23,Vienna,Austria
9,2025-11-18 00:00:00+00:00,5.85,0.0,13.202726,281.003479,61.0,46.0,35.0,60.876488,"Hausgrundweg 23, Gstr. 254",Hausgrundweg-23,Vienna,Austria


In [47]:
# Summary of predictions by sensor
print("📊 Predictions Summary by Sensor:")
print("="*80)
for sensor_street in predictions_df['street'].unique():
    sensor_df = predictions_df[predictions_df['street'] == sensor_street]
    sensor_name = sensor_df.iloc[0]['sensor_name']
    pred_values = sensor_df['predicted_pm25'].values
    
    print(f"\n{sensor_name} ({sensor_street}):")
    print(f"  Days predicted: {len(pred_values)}")
    print(f"  PM2.5 range: {pred_values.min():.1f} - {pred_values.max():.1f}")
    print(f"  Mean PM2.5: {pred_values.mean():.1f}")
    print(f"  Date range: {sensor_df['date'].min()} to {sensor_df['date'].max()}")

📊 Predictions Summary by Sensor:

Kendlerstraße 40 (Umspannwerk) (Kendlerstrasse-40):
  Days predicted: 7
  PM2.5 range: 44.0 - 65.3
  Mean PM2.5: 51.8
  Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00

Hausgrundweg 23, Gstr. 254 (Hausgrundweg-23):
  Days predicted: 7
  PM2.5 range: 57.5 - 74.3
  Mean PM2.5: 66.7
  Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00

Allgemeines Krankenhaus, Ostringweg (AKH-Ostringweg):
  Days predicted: 7
  PM2.5 range: 40.2 - 59.3
  Mean PM2.5: 49.0
  Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00

Umspannwerk Gaudenzdorfer Gürtel (Gaudenzdorfer-Guertel):
  Days predicted: 7
  PM2.5 range: 44.2 - 59.2
  Mean PM2.5: 52.2
  Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00

Belgradplatz (Südostecke), Gstr.Nr. 816 (Belgradplatz):
  Days predicted: 7
  PM2.5 range: 39.4 - 56.6
  Mean PM2.5: 49.8
  Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00

Floridsdorf, Gericht

---

## <span style="color:#ff5f27;">📊 Visualize Predictions</span>

Generate comprehensive visualizations:
1. **Individual Sensor Forecasts**: Separate plots for each sensor
2. **Multi-Sensor Comparison**: All sensors in one dashboard
3. **Forecast vs Actual**: Compare predictions with recent actual PM2.5

In [48]:
# Create output directory for images
import os
from pathlib import Path

output_dir = Path(f"{root_dir}/docs/air-quality/assets/img/vienna_forecasts")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Output directory: {output_dir}")

# ============================================================================
# VISUALIZATION 1: Multi-Sensor Comparison (All sensors in one plot)
# ============================================================================
print("\n🎨 Creating multi-sensor comparison plot...")

fig, ax = plt.subplots(figsize=(16, 10))

# Plot predictions for each sensor
colors = plt.cm.tab10(range(len(sensor_models)))

for idx, sensor_street in enumerate(predictions_df['street'].unique()):
    sensor_df = predictions_df[predictions_df['street'] == sensor_street].sort_values('date')
    sensor_name = sensor_df.iloc[0]['sensor_name']
    
    # Plot forecast line
    ax.plot(sensor_df['date'], sensor_df['predicted_pm25'], 
            marker='o', label=f'{sensor_name} (Forecast)', 
            color=colors[idx], linewidth=2, markersize=6)
    
    # Get recent actual PM2.5 for context (last 7 days)
    sensor_aq = air_quality_df[air_quality_df['street'] == sensor_street].copy()
    sensor_aq = sensor_aq.sort_values('date', ascending=False).head(7)
    sensor_aq = sensor_aq.sort_values('date')
    
    if len(sensor_aq) > 0:
        # Plot actual PM2.5 with dashed line
        ax.plot(sensor_aq['date'], sensor_aq['pm25'],
                marker='x', linestyle='--', alpha=0.6,
                color=colors[idx], linewidth=1.5, markersize=8,
                label=f'{sensor_name} (Actual)')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('PM2.5 (μg/m³)', fontsize=12)
ax.set_title('Vienna PM2.5 Forecast - All Sensors\n(Solid=Forecast, Dashed=Recent Actual)', 
             fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()

multi_sensor_path = output_dir / "multi_sensor_forecast.png"
plt.savefig(multi_sensor_path, dpi=120, bbox_inches='tight')
print(f"   ✅ Saved: {multi_sensor_path.name}")
plt.close()

print(f"\n✅ Multi-sensor comparison plot completed")

📁 Output directory: /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/assets/img/vienna_forecasts

🎨 Creating multi-sensor comparison plot...
   ✅ Saved: multi_sensor_forecast.png

✅ Multi-sensor comparison plot completed


In [49]:
# ============================================================================
# VISUALIZATION 2: Individual Sensor Plots
# ============================================================================
print("\n🎨 Creating individual sensor forecast plots...")

individual_plots = []

for idx, sensor_street in enumerate(predictions_df['street'].unique(), 1):
    sensor_df = predictions_df[predictions_df['street'] == sensor_street].sort_values('date')
    sensor_name = sensor_df.iloc[0]['sensor_name']
    
    print(f"   [{idx}/{len(predictions_df['street'].unique())}] Plotting: {sensor_name}")
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Get recent actual PM2.5 (last 14 days)
    sensor_aq = air_quality_df[air_quality_df['street'] == sensor_street].copy()
    sensor_aq = sensor_aq.sort_values('date', ascending=False).head(14)
    sensor_aq = sensor_aq.sort_values('date')
    
    if len(sensor_aq) > 0:
        # Plot actual PM2.5
        ax.plot(sensor_aq['date'], sensor_aq['pm25'],
                marker='o', linestyle='-', color='#2E86AB', linewidth=2, markersize=6,
                label='Actual PM2.5', alpha=0.8)
    
    # Plot forecast PM2.5
    ax.plot(sensor_df['date'], sensor_df['predicted_pm25'],
            marker='s', linestyle='-', color='#FF6B35', linewidth=2, markersize=6,
            label='Predicted PM2.5 (7-day forecast)')
    
    # Add vertical line to separate actual from forecast
    if len(sensor_aq) > 0:
        last_actual_date = sensor_aq['date'].max()
        ax.axvline(x=last_actual_date, color='gray', linestyle='--', alpha=0.5, linewidth=1.5,
                   label='Forecast Start')
    
    ax.set_xlabel('Date', fontsize=11)
    ax.set_ylabel('PM2.5 (μg/m³)', fontsize=11)
    ax.set_title(f'{sensor_name}\nPM2.5 Forecast vs Actual', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Save individual plot
    safe_name = sensor_street.replace('-', '_').lower()
    individual_path = output_dir / f"forecast_{safe_name}.png"
    plt.savefig(individual_path, dpi=100, bbox_inches='tight')
    individual_plots.append(str(individual_path))
    plt.close()

print(f"\n✅ Individual sensor plots completed ({len(individual_plots)} plots)")


🎨 Creating individual sensor forecast plots...
   [1/9] Plotting: Kendlerstraße 40 (Umspannwerk)
   [2/9] Plotting: Hausgrundweg 23, Gstr. 254
   [3/9] Plotting: Allgemeines Krankenhaus, Ostringweg
   [4/9] Plotting: Umspannwerk Gaudenzdorfer Gürtel
   [5/9] Plotting: Belgradplatz (Südostecke), Gstr.Nr. 816
   [6/9] Plotting: Floridsdorf, Gerichtsgasse 1a
   [7/9] Plotting: Ecke Taborstraße - Glockengasse
   [8/9] Plotting: Wehlistraße 366, Gstr.Nr.2157
   [9/9] Plotting: Schafbergbad, Josef Redl Gasse 2

✅ Individual sensor plots completed (9 plots)


---

## <span style="color:#ff5f27;">💾 Save Predictions to Feature Group</span>

Store predictions in monitoring Feature Group for future analysis and comparison.

In [50]:
# Prepare data for Feature Group
# Add metadata columns required for monitoring
monitoring_df = predictions_df.copy()

# Add days_before_forecast_day (all are 0 as these are fresh predictions made today)
monitoring_df['days_before_forecast_day'] = 0

# Sort by street and date
monitoring_df = monitoring_df.sort_values(['street', 'date'])

print(f"📊 Monitoring data prepared:")
print(f"   Total rows: {len(monitoring_df)}")
print(f"   Sensors: {monitoring_df['street'].nunique()}")
print(f"   Date range: {monitoring_df['date'].min()} to {monitoring_df['date'].max()}")

monitoring_df.head(15)

📊 Monitoring data prepared:
   Total rows: 63
   Sensors: 9
   Date range: 2025-11-16 00:00:00+00:00 to 2025-11-22 00:00:00+00:00


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,lag_1_pm25,lag_2_pm25,lag_3_pm25,predicted_pm25,sensor_name,street,city,country,days_before_forecast_day
14,2025-11-16 00:00:00+00:00,12.25,0.0,8.396570,149.036316,46.0,46.0,39.0,45.880596,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
15,2025-11-17 00:00:00+00:00,9.75,2.8,13.324863,271.548126,46.0,46.0,39.0,40.244125,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
16,2025-11-18 00:00:00+00:00,5.30,0.0,17.015474,276.072357,46.0,46.0,39.0,48.117977,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
17,2025-11-19 00:00:00+00:00,4.95,0.0,15.038350,137.910919,46.0,46.0,39.0,58.339970,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
18,2025-11-20 00:00:00+00:00,7.25,0.0,10.538843,262.146790,46.0,46.0,39.0,42.923229,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
19,2025-11-21 00:00:00+00:00,3.30,0.1,12.727921,298.739685,46.0,46.0,39.0,48.132923,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
20,2025-11-22 00:00:00+00:00,2.90,0.1,4.394360,55.007900,46.0,46.0,39.0,59.294014,"Allgemeines Krankenhaus, Ostringweg",AKH-Ostringweg,Vienna,Austria,0
28,2025-11-16 00:00:00+00:00,12.10,0.0,8.396570,149.036316,42.0,38.0,47.0,47.227619,"Belgradplatz (Südostecke), Gstr.Nr. 816",Belgradplatz,Vienna,Austria,0
29,2025-11-17 00:00:00+00:00,9.60,2.8,13.324863,271.548126,42.0,38.0,47.0,39.431431,"Belgradplatz (Südostecke), Gstr.Nr. 816",Belgradplatz,Vienna,Austria,0
30,2025-11-18 00:00:00+00:00,5.15,0.0,17.015474,276.072357,42.0,38.0,47.0,45.184475,"Belgradplatz (Südostecke), Gstr.Nr. 816",Belgradplatz,Vienna,Austria,0


In [51]:
# Get or create Vienna-specific monitoring feature group
print("\n💾 Creating/retrieving monitoring Feature Group...")

monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions_vienna',
    description='Air Quality prediction monitoring for Vienna multi-sensor system',
    version=1,
    primary_key=['city', 'street', 'date', 'days_before_forecast_day'],
    event_time="date",
    online_enabled=False,
)

print(f"✅ Feature Group: {monitor_fg.name} (v{monitor_fg.version})")


💾 Creating/retrieving monitoring Feature Group...
✅ Feature Group: aq_predictions_vienna (v1)


In [52]:
print("\n💾 Inserting predictions into Feature Group...")

# Select only the columns needed for the Feature Group
fg_columns = ['date', 'city', 'street', 'country', 'sensor_name',
              'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 
              'wind_direction_10m_dominant', 'lag_1_pm25', 'lag_2_pm25', 'lag_3_pm25',
              'predicted_pm25', 'days_before_forecast_day']

monitoring_insert_df = monitoring_df[fg_columns].copy()

print(f"   Inserting {len(monitoring_insert_df)} rows...")
monitor_fg.insert(monitoring_insert_df, wait=True)

print(f"✅ Predictions saved to Feature Group")


💾 Inserting predictions into Feature Group...
   Inserting 63 rows...


Uploading Dataframe: 100.00% |██████████| Rows 63/63 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_vienna_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1298582/jobs/named/aq_predictions_vienna_1_offline_fg_materialization/executions
2025-11-16 13:30:27,510 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-16 13:30:33,949 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-16 13:33:46,560 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-16 13:33:46,728 INFO: Waiting for log aggregation to finish.
2025-11-16 13:34:05,364 INFO: Execution finished successfully.
✅ Predictions saved to Feature Group


In [53]:
## <span style="color:#ff5f27;">📈 Hindcast Analysis</span>

# Compare previous predictions with actual PM2.5 values to evaluate model performance.

# Read monitoring data (predictions made 1 day beforehand)
print("\n📊 Retrieving hindcast data...")
print("   (Comparing predictions made 1 day ago with actual PM2.5)")

try:
    hindcast_monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
    print(f"\n✅ Retrieved {len(hindcast_monitoring_df)} hindcast rows")
    
    if len(hindcast_monitoring_df) > 0:
        print(f"   Sensors: {hindcast_monitoring_df['street'].nunique()}")
        print(f"   Date range: {hindcast_monitoring_df['date'].min()} to {hindcast_monitoring_df['date'].max()}")
    else:
        print("   ⚠️  No hindcast data yet (predictions need at least 1 day to compare)")
        
except Exception as e:
    print(f"   ⚠️  No hindcast data available yet: {str(e)}")
    hindcast_monitoring_df = pd.DataFrame()

hindcast_monitoring_df.head() if len(hindcast_monitoring_df) > 0 else None


📊 Retrieving hindcast data...
   (Comparing predictions made 1 day ago with actual PM2.5)
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.42s) 

✅ Retrieved 9 hindcast rows
   Sensors: 9
   Date range: 2025-11-18 00:00:00+00:00 to 2025-11-18 00:00:00+00:00


,date,city,street,country,sensor_name,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,lag_1_pm25,lag_2_pm25,lag_3_pm25,predicted_pm25,days_before_forecast_day
0,2025-11-18 00:00:00+00:00,Vienna,Taborstrasse,Austria,Ecke Taborstraße - Glockengasse,6.00,0.0,11.874544,284.036255,50.984703,38.0,30.0,58.139061,1
1,2025-11-18 00:00:00+00:00,Vienna,Wehlistrasse-366,Austria,"Wehlistraße 366, Gstr.Nr.2157",6.10,0.0,11.874544,284.036255,57.938576,42.0,38.0,62.328636,1
2,2025-11-18 00:00:00+00:00,Vienna,Floridsdorf-Gerichtsgasse,Austria,"Floridsdorf, Gerichtsgasse 1a",6.10,0.0,11.874544,284.036255,53.535694,46.0,40.0,55.720478,1
3,2025-11-18 00:00:00+00:00,Vienna,Belgradplatz,Austria,"Belgradplatz (Südostecke), Gstr.Nr. 816",5.45,0.0,14.830076,275.572113,40.933002,42.0,38.0,47.394264,1
4,2025-11-18 00:00:00+00:00,Vienna,Josef-Redl-Gasse,Austria,"Schafbergbad, Josef Redl Gasse 2",4.85,0.0,14.830076,275.572113,41.512596,42.0,30.0,42.802769,1


In [54]:
# Merge predictions with actual PM2.5 values
print("\n🔄 Merging predictions with actual PM2.5 values...")

if len(hindcast_monitoring_df) > 0:
    # Get actual PM2.5 from air quality feature group
    actual_pm25_df = air_quality_df[['date', 'street', 'pm25']].copy()
    actual_pm25_df = actual_pm25_df.rename(columns={'pm25': 'actual_pm25'})
    
    # Merge on date and street
    hindcast_comparison_df = pd.merge(
        hindcast_monitoring_df[['date', 'street', 'sensor_name', 'predicted_pm25']],
        actual_pm25_df,
        on=['date', 'street'],
        how='inner'
    )
    
    print(f"✅ Hindcast comparison data: {len(hindcast_comparison_df)} rows")
    
    if len(hindcast_comparison_df) > 0:
        # Calculate errors
        hindcast_comparison_df['error'] = hindcast_comparison_df['predicted_pm25'] - hindcast_comparison_df['actual_pm25']
        hindcast_comparison_df['abs_error'] = np.abs(hindcast_comparison_df['error'])
        hindcast_comparison_df['pct_error'] = (hindcast_comparison_df['error'] / hindcast_comparison_df['actual_pm25']) * 100
        
        print(f"\n📊 Hindcast Performance Summary:")
        print(f"   Mean Absolute Error: {hindcast_comparison_df['abs_error'].mean():.2f} μg/m³")
        print(f"   Mean Percentage Error: {hindcast_comparison_df['pct_error'].mean():.1f}%")
        print(f"   RMSE: {np.sqrt((hindcast_comparison_df['error']**2).mean()):.2f} μg/m³")
        
        hindcast_comparison_df.head(15)
    else:
        print("   ⚠️  No matching dates between predictions and actual PM2.5")
        hindcast_comparison_df = pd.DataFrame()
else:
    print("   ⚠️  Skipping hindcast analysis (no data)")
    hindcast_comparison_df = pd.DataFrame()


🔄 Merging predictions with actual PM2.5 values...
✅ Hindcast comparison data: 0 rows
   ⚠️  No matching dates between predictions and actual PM2.5


In [55]:
# Create hindcast visualization if data available
if len(hindcast_comparison_df) > 0:
    print("\n🎨 Creating hindcast visualization...")
    
    # Hindcast plot: Multi-sensor comparison (predicted vs actual)
    fig, ax = plt.subplots(figsize=(16, 8))
    
    colors = plt.cm.tab10(range(len(hindcast_comparison_df['street'].unique())))
    
    for idx, sensor_street in enumerate(hindcast_comparison_df['street'].unique()):
        sensor_df = hindcast_comparison_df[hindcast_comparison_df['street'] == sensor_street].sort_values('date')
        sensor_name = sensor_df.iloc[0]['sensor_name']
        
        # Plot actual vs predicted
        ax.plot(sensor_df['date'], sensor_df['actual_pm25'],
                marker='o', linestyle='-', color=colors[idx], linewidth=2, markersize=6,
                label=f'{sensor_name} (Actual)', alpha=0.8)
        ax.plot(sensor_df['date'], sensor_df['predicted_pm25'],
                marker='x', linestyle='--', color=colors[idx], linewidth=1.5, markersize=8,
                label=f'{sensor_name} (Predicted)', alpha=0.7)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('PM2.5 (μg/m³)', fontsize=12)
    ax.set_title('Vienna PM2.5 Hindcast - All Sensors\n(1-Day Ahead Predictions vs Actual)', 
                 fontsize=14, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    hindcast_path = output_dir / "multi_sensor_hindcast.png"
    plt.savefig(hindcast_path, dpi=120, bbox_inches='tight')
    print(f"   ✅ Saved: {hindcast_path.name}")
    plt.close()
    
    print(f"\n✅ Hindcast visualization completed")
else:
    print("\n⚠️  Skipping hindcast visualization (no data available yet)")
    print("   Run this notebook daily to build up hindcast data over time.")


⚠️  Skipping hindcast visualization (no data available yet)
   Run this notebook daily to build up hindcast data over time.


---

## <span style="color:#ff5f27;">📤 Upload Visualizations to Hopsworks</span>

Upload generated plots to Hopsworks for sharing and monitoring.

In [56]:
# Display all generated visualizations
print("\n📊 Generated Visualizations:")
print("="*80)

all_images = list(output_dir.glob("*.png"))
print(f"\nTotal images created: {len(all_images)}")

print(f"\n📈 Main Dashboards:")
print(f"  • multi_sensor_forecast.png - 7-day forecast for all sensors")
if (output_dir / "multi_sensor_hindcast.png").exists():
    print(f"  • multi_sensor_hindcast.png - Hindcast performance analysis")

print(f"\n📊 Individual Sensor Forecasts:")
for img_path in sorted(output_dir.glob("forecast_*.png")):
    print(f"  • {img_path.name}")

print(f"\n📁 All images saved to: {output_dir}")


📊 Generated Visualizations:

Total images created: 10

📈 Main Dashboards:
  • multi_sensor_forecast.png - 7-day forecast for all sensors

📊 Individual Sensor Forecasts:
  • forecast_akh_ostringweg.png
  • forecast_belgradplatz.png
  • forecast_floridsdorf_gerichtsgasse.png
  • forecast_gaudenzdorfer_guertel.png
  • forecast_hausgrundweg_23.png
  • forecast_josef_redl_gasse.png
  • forecast_kendlerstrasse_40.png
  • forecast_taborstrasse.png
  • forecast_wehlistrasse_366.png

📁 All images saved to: /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/assets/img/vienna_forecasts


### Upload the prediction and hindcast dashboards (png files) to Hopsworks


In [57]:
print("\n📤 Uploading visualizations to Hopsworks...")

dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")

# Create directory structure
hopsworks_dir = f"Resources/airquality_vienna/{str_today}"

if dataset_api.exists("Resources/airquality_vienna") == False:
    dataset_api.mkdir("Resources/airquality_vienna")

print(f"   Target directory: {hopsworks_dir}")

# Upload all images
uploaded_count = 0
for img_path in output_dir.glob("*.png"):
    try:
        dataset_api.upload(str(img_path), hopsworks_dir, overwrite=True)
        uploaded_count += 1
        print(f"   ✅ Uploaded: {img_path.name}")
    except Exception as e:
        print(f"   ❌ Failed to upload {img_path.name}: {str(e)}")

proj_url = project.get_url()
print(f"\n✅ Uploaded {uploaded_count} images to Hopsworks")
print(f"\n🔗 View images here: {proj_url}/settings/fb/path/Resources/airquality_vienna")


📤 Uploading visualizations to Hopsworks...
   Target directory: Resources/airquality_vienna/2025-11-15


Uploading /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/asse…

   ✅ Uploaded: forecast_taborstrasse.png


Uploading /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/asse…

   ✅ Uploaded: forecast_akh_ostringweg.png


...

   ✅ Uploaded: forecast_floridsdorf_gerichtsgasse.png


Uploading /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/asse…

   ✅ Uploaded: forecast_belgradplatz.png


Uploading /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction/docs/air-quality/asse…

   ✅ Uploaded: multi_sensor_forecast.png

✅ Uploaded 10 images to Hopsworks

🔗 View images here: https://c.app.hopsworks.ai:443/p/1298582/settings/fb/path/Resources/airquality_vienna


---

## <span style="color:#ff5f27;">🎉 Pipeline Summary</span>

### ✅ Completed Tasks:

1. **Models Loaded**: Retrieved and loaded 9 XGBoost models from Hopsworks Model Registry
2. **Features Prepared**: Fetched weather forecasts and lagged PM2.5 features for all sensors
3. **Predictions Generated**: Created 7-day PM2.5 forecasts for each sensor
4. **Visualizations Created**:
   - Multi-sensor forecast dashboard (all sensors in one plot)
   - Individual sensor forecast plots (9 plots)
   - Hindcast performance analysis (when available)
5. **Data Stored**: Saved predictions to monitoring Feature Group
6. **Results Uploaded**: Uploaded all visualizations to Hopsworks

### 📊 Next Steps:

- **Daily Execution**: Run this notebook daily to build up hindcast data
- **GitHub Pages**: Display visualizations on your project GitHub Pages
- **Monitoring**: Track model performance over time using hindcast analysis
- **Improvements**: Fine-tune models based on performance metrics

### 🔗 Useful Links:

- **Hopsworks Dashboard**: View Feature Groups and models
- **Model Registry**: Check model versions and metrics
- **Visualizations**: Access all generated plots in Hopsworks Resources

---

**🎯 Vienna Multi-Sensor PM2.5 Prediction System**  
*Providing 7-day air quality forecasts for 9 sensors across Vienna*